<a href="https://colab.research.google.com/github/Paaaark/Kaggle/blob/main/titanic/helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing
import sklearn
import csv
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def process_for_graphs(df):
  # Fill nan values with 0
  for key in df.keys():
    df[key] = df[key].fillna(0)

  survival_rate = {}
  for key in df.keys():
    if key in ['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin']:
      continue
    survival_rate[key] = {};
    for ind in df.index:
      if df[key][ind] in survival_rate[key]:
        survival_rate[key][df[key][ind]][df['Survived'][ind]] += 1
      else:
        survival_rate[key][df[key][ind]] = [0, 0]
        survival_rate[key][df[key][ind]][df['Survived'][ind]] += 1

  age_count = {}
  for i in range(0, 80, 10):
    age_count[i] = [0, 0]
  for entry in survival_rate['Age'].keys():
    age = int(entry / 10) * 10
    if age > 70:
      age = 70
    age_count[age][0] += survival_rate['Age'][entry][0]
    age_count[age][1] += survival_rate['Age'][entry][1]
  survival_rate['Age'] = age_count

  fare_count = {}
  for i in range(0, 120, 20):
    fare_count[i] = [0, 0]
  for entry in survival_rate['Fare'].keys():
    fare = int(entry / 20) * 20
    if fare > 100:
      fare = 100
    fare_count[fare][0] += survival_rate['Fare'][entry][0]
    fare_count[fare][1] += survival_rate['Fare'][entry][1]
  survival_rate['Fare'] = fare_count

  return survival_rate

In [ ]:
 def graph_data(survival_rate):
  fix, axes = plt.subplots(4, 2, figsize=(10, 9))
  keys = survival_rate.keys()
  counter = 0
  for key in keys:
    ax = axes[int(counter / 2)][counter % 2]
    x_axis = survival_rate[key].keys()
    # y_axis_1 = not survived, y_axis_2 = survived
    y_axis_1 = []
    y_axis_2 = []
    label_1 = []
    label_2 = []
    for x in x_axis:
      total = float(survival_rate[key][x][0] + survival_rate[key][x][1])
      y_axis_1.append(survival_rate[key][x][0])
      y_axis_2.append(survival_rate[key][x][1])
      label_1.append(round(survival_rate[key][x][0] / total, 2))
      label_2.append(round(survival_rate[key][x][1] / total, 2))

    x_axis = list(map(lambda x: str(x), survival_rate[key].keys()))
    bar1 = ax.bar(x_axis, y_axis_1, color='red', width=0.5)
    bar2 = ax.bar(x_axis, y_axis_2, bottom=y_axis_1, color='green', width=0.5)
    ax.bar_label(bar1, label_1, label_type='center')
    ax.bar_label(bar2, label_2, label_type='center')
    ax.set_title(key)
    counter = counter + 1

In [ ]:
def process_data(data, split_rate=None, to_tensor=True, drop_columns=None):
  train_data = data
  train_data = train_data.drop(axis=1, labels=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age'])

  # age = list(train_data.loc[:, 'Age'])
  # for i in range(len(age)):
  #   age[i] = min(age[i], 70)
  # train_data.loc[:, 'Age'] = age

  fare = list(train_data.loc[:, 'Fare'])
  for i in range(len(fare)):
    fare[i] = min(fare[i], 100)
  train_data.loc[:, 'Fare'] = fare
  train_data['Fare'] = train_data['Fare'].fillna(0.0)

  train_data['Sex'] = train_data['Sex'].map({
      'male': 0,
      'female': 1
  })
  train_data['Embarked'] = train_data['Embarked'].map({
      'S': 0,
      'C': 1,
      'Q': 2,
      0 : 3,
  })
  train_data['Embarked'] = train_data['Embarked'].fillna(4)

  train_data['Fare'] = (train_data['Fare'] - train_data['Fare'].mean()) / train_data['Fare'].std()

  if drop_columns is not None:
    for col in drop_columns:
      train_data = train_data.drop(col, axis=1)

  try:
    train_y = train_data.pop('Survived')
  except KeyError:
    train_x = tf.convert_to_tensor(train_data)
    return train_x

  if split_rate is not None:
    train_x, test_x, train_y, test_y = model_selection.train_test_split(train_data, train_y, test_size=split_rate)
  else:
    if to_tensor:
      train_data = tf.convert_to_tensor(train_data)
    return train_data, train_y

  if to_tensor:
    train_x = tf.convert_to_tensor(train_x)
    test_x = tf.convert_to_tensor(test_x)
  # scaler = preprocessing.StandardScaler().fit(train_x)
  # train_x = scaler.transform(train_x)


  return train_x, test_x, train_y, test_y

In [ ]:
def save_model(model, acc, name):
  model.save('/content/drive/MyDrive/Kaggle/titanic/models/' + name + '.keras')
  plt.plot(acc)
  plt.title(name)
  plt.savefig('/content/drive/MyDrive/Kaggle/titanic/accuracy/' + name + '.jpg')

In [ ]:
def write_prediction(prediction, name):
  with open('/content/drive/MyDrive/Kaggle/titanic/submissions/' + name + '.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['PassengerId', 'Survived'])
    for i in range(len(prediction)):
      currentRow = [892 + i, int(prediction[i])]
      writer.writerow(currentRow)